In [1]:
import numpy as np 
import pandas as pd

# Data Preparation

In [2]:
bd=pd.read_csv("/kaggle/input/final-training/training_data.layer_two_bot.csv")
bd['is_bot'] ="True"
hd=pd.read_csv("/kaggle/input/final-training/training_data.layer_two_human.csv")
hd["is_bot"]="False"

In [3]:
hd["is_bot"]

0      False
1      False
2      False
3      False
4      False
       ...  
204    False
205    False
206    False
207    False
208    False
Name: is_bot, Length: 209, dtype: object

In [4]:
data= pd.concat([hd, bd], ignore_index=True)

In [5]:
data.columns

Index(['_id', 'is_abuser', 'time_taken', 'mouse_distance', 'is_proxy', 'state',
       'country', 'is_solved', 'user_agent', 'is_bot'],
      dtype='object')

In [6]:
data["is_proxy"]="false"

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1265 entries, 0 to 1264
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   _id             1265 non-null   object 
 1   is_abuser       1265 non-null   bool   
 2   time_taken      1265 non-null   float64
 3   mouse_distance  1265 non-null   float64
 4   is_proxy        1265 non-null   object 
 5   state           1265 non-null   object 
 6   country         1265 non-null   object 
 7   is_solved       1265 non-null   bool   
 8   user_agent      1265 non-null   object 
 9   is_bot          1265 non-null   object 
dtypes: bool(2), float64(2), object(6)
memory usage: 81.7+ KB


In [8]:
data["is_bot"]

0       False
1       False
2       False
3       False
4       False
        ...  
1260     True
1261     True
1262     True
1263     True
1264     True
Name: is_bot, Length: 1265, dtype: object

In [9]:
!pip install pyyaml ua-parser user-agents

In [10]:
from user_agents import parse

In [11]:
data["user_agent"]

0       Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
1       Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
2       Mozilla/5.0 (Linux; Android 10; K) AppleWebKit...
3       Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
4       Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
                              ...                        
1260    Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...
1261    Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...
1262    Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...
1263    Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...
1264    Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...
Name: user_agent, Length: 1265, dtype: object

In [12]:
def parse_user_agent(ua_string):
    ua = parse(ua_string)
    return {
        'browser_family': ua.browser.family,
        'browser_version': ua.browser.version_string,
        'os_family': ua.os.family,
        'os_version': ua.os.version_string,
        'device_family': ua.device.family,
        'is_mobile': ua.is_mobile,
        'is_tablet': ua.is_tablet,
        'is_pc': ua.is_pc,
        
    }

user_agent_features = data['user_agent'].apply(parse_user_agent).apply(pd.Series)
data = pd.concat([data, user_agent_features], axis=1)
data = data.drop(columns=['user_agent'])

In [13]:
data

,_id,is_abuser,time_taken,mouse_distance,is_proxy,state,country,is_solved,is_bot,browser_family,browser_version,os_family,os_version,device_family,is_mobile,is_tablet,is_pc
0,66f7b99e22c5ebac6db3e9d7,False,4.649,1093.685516,false,Delhi,India,True,False,Chrome,129.0.0,Windows,10,Other,False,False,True
1,66f7b99e22c5ebac6db3e9d9,False,1.806,686.580286,false,Delhi,India,True,False,Chrome,129.0.0,Windows,10,Other,False,False,True
2,66f7bce822c5ebac6db3e9db,True,5.751,318.844492,false,Delhi,India,True,False,Chrome Mobile,128.0.0,Android,10,K,True,False,False
3,66f7c03822c5ebac6db3e9de,False,9.690,2391.626054,false,Delhi,India,True,False,Chrome,128.0.0,Windows,10,Other,False,False,True
4,66f7c04822c5ebac6db3e9e1,False,3.280,1634.520909,false,Delhi,India,True,False,Chrome,128.0.0,Windows,10,Other,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260,66f79be66e961372ec4a6efc,False,1.231,820.088537,false,Haryana,India,True,True,Chrome,128.0.0,Linux,,Other,False,False,True
1261,66f79bfa6e961372ec4a6eff,False,2.705,286.005000,false,Haryana,India,False,True,Chrome,128.0.0,Linux,,Other,False,False,True
1262,66f79bfc6e961372ec4a6f00,False,1.199,636.282976,false,Haryana,India,False,True,Chrome,128.0.0,Linux,,Other,False,False,True
1263,66f79bfd6e961372ec4a6f01,False,1.164,979.832554,false,Haryana,India,True,True,Chrome,128.0.0,Linux,,Other,False,False,True


In [14]:
#data["typing_speed"].fillna(0)

In [15]:
data

,_id,is_abuser,time_taken,mouse_distance,is_proxy,state,country,is_solved,is_bot,browser_family,browser_version,os_family,os_version,device_family,is_mobile,is_tablet,is_pc
0,66f7b99e22c5ebac6db3e9d7,False,4.649,1093.685516,false,Delhi,India,True,False,Chrome,129.0.0,Windows,10,Other,False,False,True
1,66f7b99e22c5ebac6db3e9d9,False,1.806,686.580286,false,Delhi,India,True,False,Chrome,129.0.0,Windows,10,Other,False,False,True
2,66f7bce822c5ebac6db3e9db,True,5.751,318.844492,false,Delhi,India,True,False,Chrome Mobile,128.0.0,Android,10,K,True,False,False
3,66f7c03822c5ebac6db3e9de,False,9.690,2391.626054,false,Delhi,India,True,False,Chrome,128.0.0,Windows,10,Other,False,False,True
4,66f7c04822c5ebac6db3e9e1,False,3.280,1634.520909,false,Delhi,India,True,False,Chrome,128.0.0,Windows,10,Other,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260,66f79be66e961372ec4a6efc,False,1.231,820.088537,false,Haryana,India,True,True,Chrome,128.0.0,Linux,,Other,False,False,True
1261,66f79bfa6e961372ec4a6eff,False,2.705,286.005000,false,Haryana,India,False,True,Chrome,128.0.0,Linux,,Other,False,False,True
1262,66f79bfc6e961372ec4a6f00,False,1.199,636.282976,false,Haryana,India,False,True,Chrome,128.0.0,Linux,,Other,False,False,True
1263,66f79bfd6e961372ec4a6f01,False,1.164,979.832554,false,Haryana,India,True,True,Chrome,128.0.0,Linux,,Other,False,False,True


In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow.keras import layers
import joblib
label_enc = LabelEncoder()
data['country'] = label_enc.fit_transform(data['country'])
joblib.dump(label_enc, 'country_label_encoder.joblib')
label_enc = LabelEncoder()
data['state'] = label_enc.fit_transform(data['state'])
joblib.dump(label_enc, 'city_label_encoder.joblib')
label_enc = LabelEncoder()
data['is_proxy'] = label_enc.fit_transform(data['is_proxy'])
joblib.dump(label_enc, 'proxy_label_encoder.joblib')
label_enc = LabelEncoder()
data['is_abuser'] = label_enc.fit_transform(data['is_abuser'])
joblib.dump(label_enc, 'abuser_label_encoder.joblib')
label_enc = LabelEncoder()

for column in ['browser_family', 'os_family', 'device_family','browser_version','os_version']:
    label_enc = LabelEncoder()
    data[column] = label_enc.fit_transform(data[column])
    joblib.dump(label_enc, f'{column}_label_encoder.joblib')
   

data['is_mobile'] = data['is_mobile'].astype(int)
data['is_tablet'] = data['is_tablet'].astype(int)
data['is_pc'] = data['is_pc'].astype(int)


In [17]:
data

,_id,is_abuser,time_taken,mouse_distance,is_proxy,state,country,is_solved,is_bot,browser_family,browser_version,os_family,os_version,device_family,is_mobile,is_tablet,is_pc
0,66f7b99e22c5ebac6db3e9d7,0,4.649,1093.685516,0,0,0,True,False,0,3,3,1,2,0,0,1
1,66f7b99e22c5ebac6db3e9d9,0,1.806,686.580286,0,0,0,True,False,0,3,3,1,2,0,0,1
2,66f7bce822c5ebac6db3e9db,1,5.751,318.844492,0,0,0,True,False,1,2,0,1,0,1,0,0
3,66f7c03822c5ebac6db3e9de,0,9.690,2391.626054,0,0,0,True,False,0,2,3,1,2,0,0,1
4,66f7c04822c5ebac6db3e9e1,0,3.280,1634.520909,0,0,0,True,False,0,2,3,1,2,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260,66f79be66e961372ec4a6efc,0,1.231,820.088537,0,1,0,True,True,0,2,1,0,2,0,0,1
1261,66f79bfa6e961372ec4a6eff,0,2.705,286.005000,0,1,0,False,True,0,2,1,0,2,0,0,1
1262,66f79bfc6e961372ec4a6f00,0,1.199,636.282976,0,1,0,False,True,0,2,1,0,2,0,0,1
1263,66f79bfd6e961372ec4a6f01,0,1.164,979.832554,0,1,0,True,True,0,2,1,0,2,0,0,1


In [18]:
len(data["mouse_distance"].unique())

1129

In [19]:
data=data.drop(["_id","is_proxy"],axis=1)

In [20]:
data

,is_abuser,time_taken,mouse_distance,state,country,is_solved,is_bot,browser_family,browser_version,os_family,os_version,device_family,is_mobile,is_tablet,is_pc
0,0,4.649,1093.685516,0,0,True,False,0,3,3,1,2,0,0,1
1,0,1.806,686.580286,0,0,True,False,0,3,3,1,2,0,0,1
2,1,5.751,318.844492,0,0,True,False,1,2,0,1,0,1,0,0
3,0,9.690,2391.626054,0,0,True,False,0,2,3,1,2,0,0,1
4,0,3.280,1634.520909,0,0,True,False,0,2,3,1,2,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260,0,1.231,820.088537,1,0,True,True,0,2,1,0,2,0,0,1
1261,0,2.705,286.005000,1,0,False,True,0,2,1,0,2,0,0,1
1262,0,1.199,636.282976,1,0,False,True,0,2,1,0,2,0,0,1
1263,0,1.164,979.832554,1,0,True,True,0,2,1,0,2,0,0,1


In [21]:
data['is_bot'] = data['is_bot'].map({"True":1,"False":0})

X = data.drop('is_bot', axis=1)
y = data['is_bot']

In [22]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1265 entries, 0 to 1264
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   is_abuser        1265 non-null   int64  
 1   time_taken       1265 non-null   float64
 2   mouse_distance   1265 non-null   float64
 3   state            1265 non-null   int64  
 4   country          1265 non-null   int64  
 5   is_solved        1265 non-null   bool   
 6   browser_family   1265 non-null   int64  
 7   browser_version  1265 non-null   int64  
 8   os_family        1265 non-null   int64  
 9   os_version       1265 non-null   int64  
 10  device_family    1265 non-null   int64  
 11  is_mobile        1265 non-null   int64  
 12  is_tablet        1265 non-null   int64  
 13  is_pc            1265 non-null   int64  
dtypes: bool(1), float64(2), int64(11)
memory usage: 129.8 KB


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1265 entries, 0 to 1264
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   is_abuser        1265 non-null   int64  
 1   time_taken       1265 non-null   float64
 2   mouse_distance   1265 non-null   float64
 3   state            1265 non-null   int64  
 4   country          1265 non-null   int64  
 5   is_solved        1265 non-null   bool   
 6   is_bot           1265 non-null   int64  
 7   browser_family   1265 non-null   int64  
 8   browser_version  1265 non-null   int64  
 9   os_family        1265 non-null   int64  
 10  os_version       1265 non-null   int64  
 11  device_family    1265 non-null   int64  
 12  is_mobile        1265 non-null   int64  
 13  is_tablet        1265 non-null   int64  
 14  is_pc            1265 non-null   int64  
dtypes: bool(1), float64(2), int64(12)
memory usage: 139.7 KB


# Splitting

In [24]:
y

0       0
1       0
2       0
3       0
4       0
       ..
1260    1
1261    1
1262    1
1263    1
1264    1
Name: is_bot, Length: 1265, dtype: int64

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

In [26]:
correlation_matrix = data.corr()
print(correlation_matrix)


                 is_abuser  time_taken  mouse_distance     state  country  \
is_abuser         1.000000    0.062154       -0.050190 -0.182165      NaN   
time_taken        0.062154    1.000000       -0.225753 -0.056729      NaN   
mouse_distance   -0.050190   -0.225753        1.000000  0.023683      NaN   
state            -0.182165   -0.056729        0.023683  1.000000      NaN   
country                NaN         NaN             NaN       NaN      NaN   
is_solved         0.101035    0.078520       -0.035482 -0.166552      NaN   
is_bot           -0.254412   -0.224386       -0.091292  0.467871      NaN   
browser_family    0.307471    0.070597       -0.114726 -0.168001      NaN   
browser_version   0.014173   -0.005750       -0.056125 -0.788893      NaN   
os_family        -0.086000    0.152381        0.170773 -0.645685      NaN   
os_version        0.096484    0.137782        0.016770 -0.742755      NaN   
device_family    -0.292427   -0.080960        0.113408  0.442999      NaN   

# XGB training

In [27]:
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
xgb_model.save_model('demo_weights_xgboost.json')
xgb_preds = xgb_model.predict(X_test)
print(xgb_preds)
print("XGBoost Accuracy:", accuracy_score(y_test, xgb_preds))
print(classification_report(y_test, xgb_preds))

[1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
 0 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 0 1 0 1 1 1 0 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1
 1 0 1 1 1 1 0 0 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1
 0 1 1 0 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 0 1 1 1 0 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 0]
XGBoost Accuracy: 0.9747634069400631
              precision    recall  f1-score   support

           0       0.92      0.92      0.92        52
           1       0.98      0.98      0.98       265

    accuracy                           0.97       317
   macro avg       0.95      0.95      0.95       317
weighted avg       0.97      0.97      0.97    

# CatBoost Training

In [28]:
catboost_model = CatBoostClassifier(verbose=0)
catboost_model.fit(X_train, y_train)
catboost_model.save_model('demo_weights_catboost.json')
catboost_preds = catboost_model.predict(X_test)
print("CatBoost Accuracy:", accuracy_score(y_test, catboost_preds))
print(classification_report(y_test, catboost_preds))

CatBoost Accuracy: 0.9810725552050473
              precision    recall  f1-score   support

           0       0.94      0.94      0.94        52
           1       0.99      0.99      0.99       265

    accuracy                           0.98       317
   macro avg       0.97      0.97      0.97       317
weighted avg       0.98      0.98      0.98       317



# Neural Network Based approach

In [29]:
"""neural_model=[]
nueral_model=tf.keras.Sequential([
        layers.InputLayer(input_shape=(X.shape[1],)),
        layers.Dense(34),
        layers.Dense(32, activation='relu'),
        layers.Dense(8),
        layers.Dense(1, activation='sigmoid')  
    ])
nueral_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
nueral_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)
nueral_model.save_weights("demo_weights_neural.weights.h5")
nueral_model_preds=nueral_model.predict(X_test)
nueral_model_preds_test = (nueral_model_preds > 0.5).astype(int) 

print("Neural Network Accuracy:", accuracy_score(y_test, nueral_model_preds_test))
print(classification_report(y_test, nueral_model_preds_test))"""

'neural_model=[]\nnueral_model=tf.keras.Sequential([\n        layers.InputLayer(input_shape=(X.shape[1],)),\n        layers.Dense(34),\n        layers.Dense(32, activation=\'relu\'),\n        layers.Dense(8),\n        layers.Dense(1, activation=\'sigmoid\')  \n    ])\nnueral_model.compile(optimizer=\'adam\', loss=\'binary_crossentropy\', metrics=[\'accuracy\'])\nnueral_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)\nnueral_model.save_weights("demo_weights_neural.weights.h5")\nnueral_model_preds=nueral_model.predict(X_test)\nnueral_model_preds_test = (nueral_model_preds > 0.5).astype(int) \n\nprint("Neural Network Accuracy:", accuracy_score(y_test, nueral_model_preds_test))\nprint(classification_report(y_test, nueral_model_preds_test))'

In [30]:
import lightgbm as lgb
lgbm_model = lgb.LGBMClassifier()
lgbm_model.fit(X_train, y_train)
lgbm_model.booster_.save_model('demo_weights_lgbm.txt')
joblib.dump(lgbm_model, 'lgbm_model.joblib')
lgbm_preds = lgbm_model.predict(X_test)
print(lgbm_preds)
print("LightGBM Accuracy:", accuracy_score(y_test, lgbm_preds))
print(classification_report(y_test, lgbm_preds))

[LightGBM] [Info] Number of positive: 791, number of negative: 157
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002447 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 541
[LightGBM] [Info] Number of data points in the train set: 948, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.834388 -> initscore=1.617052
[LightGBM] [Info] Start training from score 1.617052
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

# Mixture of Experts(MoE) Approach

In [31]:
xgb_preds_proba_train = xgb_model.predict_proba(X_train)[:, 1]  
catboost_preds_proba_train = catboost_model.predict_proba(X_train)[:, 1]
lgbm_model_preds_proba_train = lgbm_model.predict_proba(X_train)[:, 1] 

xgb_preds_proba_test = xgb_model.predict_proba(X_test)[:, 1]
catboost_preds_proba_test = catboost_model.predict_proba(X_test)[:, 1]
lgbm_model_preds_proba_test = lgbm_model.predict_proba(X_test)[:, 1] 

train_expert_predictions = np.column_stack((xgb_preds_proba_train, catboost_preds_proba_train,lgbm_model_preds_proba_train))
test_expert_predictions = np.column_stack((xgb_preds_proba_test, catboost_preds_proba_test,lgbm_model_preds_proba_test))


In [32]:
def build_gate_model():
    model = tf.keras.Sequential([
        layers.InputLayer(input_shape=(3,)),
        layers.Dense(32),
        layers.Dense(16, activation='relu'),
        layers.Dense(8),
        layers.Dense(1, activation='sigmoid')  
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

gate_model = build_gate_model()

gate_model.fit(train_expert_predictions, y_train, epochs=20, batch_size=32, verbose=0)
gate_model.save_weights("demo_weights_gate.weights.h5")
gate_preds_proba_test = gate_model.predict(test_expert_predictions)
gate_preds_test = (gate_preds_proba_test > 0.5).astype(int) 

print("Neural Network Gate Model (MoE) Accuracy:", accuracy_score(y_test, gate_preds_test))
print(classification_report(y_test, gate_preds_test))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Neural Network Gate Model (MoE) Accuracy: 0.9779179810725552
              precision    recall  f1-score   support

           0       0.92      0.94      0.93        52
           1       0.99      0.98      0.99       265

    accuracy                           0.98       317
   macro avg       0.96      0.96      0.96       317
weighted avg       0.98      0.98      0.98       317

